In [1]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: list(map(str, path.iterdir()))

sys.path.append('../src')
mturk_path = Path("../mturk/results")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

%load_ext autoreload
%autoreload 2

In [2]:
ls(mturk_path)

['../mturk/results/Batch_4258572_batch_results.csv']

In [3]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
    return df.set_index("AssignmentId")
batch = 4258572
df=load_results(batch)
df

,HITId,HITTypeId,WorkerId,events,feedback,id,status
AssignmentId,,,,,,,
31QTRG6Q2Z8V7XMOTUWVQHF1WUEPY2,3O2Y2UIUDVPM9VBJLAUYGVQZ34IFKE,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A3KU8DC7HJOGCM,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},17641,1
3GD6L00D3YSSWOE8PPXUVQAWK6CM1R,3O2Y2UIUDVPM9VBJLAUYGVQZ34IFKE,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A2SBN9B6Q3DN2X,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},17641,1
3SUWZRL0M48PRMNSEXLSDEW4JAA6EJ,3O2Y2UIUDVPM9VBJLAUYGVQZ34IFKE,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A1AYL5V5GI9HC1,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},17641,1
36AHBNMV1X7MI0JYOL50QNYCICKYD8,3MNJFORX9GZJRCQGENQSE67H6EN5FJ,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A3KU8DC7HJOGCM,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},228705,1
3DZQRBDBSRAR4M5SWQ7ZVSOZBI23SD,3MNJFORX9GZJRCQGENQSE67H6EN5FJ,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A2SBN9B6Q3DN2X,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},228705,1
3YDTZAI2W3BCSYP2JLXY9WTWYA841L,3MNJFORX9GZJRCQGENQSE67H6EN5FJ,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A1AYL5V5GI9HC1,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},228705,1
320DUZ38GDHZZVYQ4B55MLMLILKGJ5,3BVS8WK9R5Q5QJ9FG5NQKPK1W21IBO,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,AORHXBTOCXFUK,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",thermometer,223296,-1
33ISQZVXPVHZA5CFSXS09QOBUKBCCR,3BVS8WK9R5Q5QJ9FG5NQKPK1W21IBO,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A1AYL5V5GI9HC1,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},223296,1
3GNCZX450OIUBFOLDOIFNVXUTKAAPA,3BVS8WK9R5Q5QJ9FG5NQKPK1W21IBO,3VGR6KHJE749DD8NO6GZS1D2TV0EYE,A3BMEVFSETVOUR,"[[{""start"":""2020-01-01T00:00:00"",""end"":""2020-0...",{},223296,1


In [6]:
html=""
for assignment, row in df.iterrows():
    idx = row["id"]
    status = row["status"]
    html+=f'<a href="http://127.0.0.1:8080/annotate/{batch}/{assignment}" target="_new">{idx} - {status}</a><br />'
display_html(html)

In [33]:
str(next(mturk_path.rglob("*4258572*.csv")))

'../mturk/results/Batch_4258572_batch_results.csv'